In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE

In [ ]:
data = pd.read_csv('/content/predictive_maintenance.csv')

In [ ]:
# Data Preprocessing
# Normalize numerical features
scaler = StandardScaler()
numerical_features = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

In [ ]:
# Encode categorical features
label_encoder = LabelEncoder()
data['Type'] = label_encoder.fit_transform(data['Type'])
data['Failure Type'] = label_encoder.fit_transform(data['Failure Type'])


In [ ]:
# Drop 'Product ID' column since it's non-numeric

data = data.drop(['Product ID', 'UDI'], axis=1)


In [ ]:
# Verify all columns are numeric
print(data.dtypes)

Type                         int64
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]     float64
Torque [Nm]                float64
Tool wear [min]            float64
Target                       int64
Failure Type                 int64
dtype: object


In [ ]:
# Split the data into features and target
X = data.drop(['Target', 'Failure Type'], axis=1)
y = data['Target']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Data Augmentation using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
# Reshape data for LSTM (samples, time steps, features)
X_train_res = X_train_res.values.reshape((X_train_res.shape[0], 1, X_train_res.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Build LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_res.shape[1], X_train_res.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train_res, y_train_res, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/50
387/387 [==============================] - 5s 4ms/step - loss: 0.4662 - accuracy: 0.8148 - val_loss: 0.3550 - val_accuracy: 0.8472
Epoch 2/50
387/387 [==============================] - 1s 4ms/step - loss: 0.3141 - accuracy: 0.8672 - val_loss: 0.3390 - val_accuracy: 0.8550
Epoch 3/50
387/387 [==============================] - 1s 3ms/step - loss: 0.2858 - accuracy: 0.8826 - val_loss: 0.2825 - val_accuracy: 0.8903
Epoch 4/50
387/387 [==============================] - 1s 3ms/step - loss: 0.2653 - accuracy: 0.8887 - val_loss: 0.2778 - val_accuracy: 0.8964
Epoch 5/50
387/387 [==============================] - 1s 3ms/step - loss: 0.2523 - accuracy: 0.8989 - val_loss: 0.2954 - val_accuracy: 0.8909
Epoch 6/50
387/387 [==============================] - 1s 3ms/step - loss: 0.2402 - accuracy: 0.9044 - val_loss: 0.2468 - val_accuracy: 0.9123
Epoch 7/50
387/387 [==============================] - 1s 3ms/step - loss: 0.2325 - accuracy: 0.9071 - val_loss: 0.2866 - val_accuracy: 0.8967
Epoch 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

63/63 [==============================] - 0s 2ms/step - loss: 0.1501 - accuracy: 0.9360
Test Loss: 0.15010680258274078
Test Accuracy: 0.9359999895095825


In [ ]:
model.save('pretrained_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Transfer Learning (fine-tuning a pre-trained model)
# Assuming a pre-trained model 'pretrained_model.h5' is available
pretrained_model = tf.keras.models.load_model('pretrained_model.h5')

In [ ]:


# Fine-tune the model
for layer in pretrained_model.layers[:-1]:
    layer.trainable = False

pretrained_model.add(Dense(1, activation='sigmoid'))

pretrained_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_transfer = pretrained_model.fit(X_train_res, y_train_res, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the fine-tuned model
loss_transfer, accuracy_transfer = pretrained_model.evaluate(X_test, y_test)
print(f'Test Loss (Transfer Learning): {loss_transfer}')
print(f'Test Accuracy (Transfer Learning): {accuracy_transfer}')

Epoch 1/50
387/387 [==============================] - 7s 9ms/step - loss: 0.6501 - accuracy: 0.8605 - val_loss: 0.6727 - val_accuracy: 0.8805
Epoch 2/50
387/387 [==============================] - 3s 7ms/step - loss: 0.5738 - accuracy: 0.9348 - val_loss: 0.6253 - val_accuracy: 0.9132
Epoch 3/50
387/387 [==============================] - 1s 3ms/step - loss: 0.5106 - accuracy: 0.9401 - val_loss: 0.5694 - val_accuracy: 0.9291
Epoch 4/50
387/387 [==============================] - 1s 3ms/step - loss: 0.4581 - accuracy: 0.9412 - val_loss: 0.5192 - val_accuracy: 0.9336
Epoch 5/50
387/387 [==============================] - 1s 3ms/step - loss: 0.4139 - accuracy: 0.9392 - val_loss: 0.4711 - val_accuracy: 0.9404
Epoch 6/50
387/387 [==============================] - 1s 3ms/step - loss: 0.3772 - accuracy: 0.9415 - val_loss: 0.4325 - val_accuracy: 0.9421
Epoch 7/50
387/387 [==============================] - 1s 3ms/step - loss: 0.3453 - accuracy: 0.9388 - val_loss: 0.3917 - val_accuracy: 0.9450
Epoch 